In [1]:
###显微镜设备自检测试
import time
Version = 'V2' ###V1
if Version == 'V1':
    from Drives.microcontroller_V1 import Microcontroller
    import control.core_V1 as core
    from Drives._def_V1 import *
elif Version == 'V2':
    from Drives.microcontroller_V2 import Microcontroller
    import control.core_V2 as core
    from Drives._def_V2 import *
# 初始化显微镜控制器
microcontroller = Microcontroller(COM='COM1')
# 重置显微镜控制器
microcontroller.reset()
# 初始化马达驱动
microcontroller.initialize_drivers()
# 配置执行器
microcontroller.configure_actuators()
print(显微镜初始化成功)

In [2]:
# 初始化导航控制器
navigationController = core.NavigationController(microcontroller)
# 将Z轴归零
navigationController.home_z()
# 等待Z轴归零完成
t0 = time.time()
while microcontroller.is_busy():
    time.sleep(0.005)
    if time.time() - t0 > 8:
        print('Z轴归零超时')
        break

In [ ]:
# 将X轴归零
t0 = time.time()
navigationController.home_x()
# 等待X轴归零完成
while microcontroller.is_busy():
    time.sleep(0.005)
    if time.time() - t0 > 8:
        print('X轴归零超时')
        break

In [3]:
# 将Y轴归零
t0 = time.time()
navigationController.home_y()
# 等待Y轴归零完成
while microcontroller.is_busy():
    time.sleep(0.005)
    if time.time() - t0 > 8:
        print('Y轴归零超时')
        break

In [ ]:
### 配置PID设置
if Version == 'V1':
    # 根据配置信息，条件性地配置编码器
    if HAS_ENCODER_X == True:
        navigationController.configure_encoder(0, (SCREW_PITCH_X_MM * 1000) / ENCODER_RESOLUTION_UM_X,
                                                            ENCODER_FLIP_DIR_X)
    if HAS_ENCODER_Y == True:
        navigationController.configure_encoder(1, (SCREW_PITCH_Y_MM * 1000) / ENCODER_RESOLUTION_UM_Y,
                                                            ENCODER_FLIP_DIR_Y)
    if HAS_ENCODER_Z == True:
        navigationController.configure_encoder(2, (SCREW_PITCH_Z_MM * 1000) / ENCODER_RESOLUTION_UM_Z,
                                                            ENCODER_FLIP_DIR_Z)
    # 启用PID控制
    navigationController.set_pid_control_enable(0, ENABLE_PID_X)
    navigationController.set_pid_control_enable(1, ENABLE_PID_Y)
    navigationController.set_pid_control_enable(2, ENABLE_PID_Z)
    time.sleep(0.5)
elif Version == 'V2':
    # 根据配置信息，条件性地配置编码器
    if HAS_ENCODER_X == True:
        navigationController.set_axis_PID_arguments(0, PID_P_X, PID_I_X, PID_D_X)
        navigationController.configure_encoder(0, (SCREW_PITCH_X_MM * 1000) / ENCODER_RESOLUTION_UM_X,
                                                            ENCODER_FLIP_DIR_X)
        navigationController.set_pid_control_enable(0, ENABLE_PID_X)
        if HAS_ENCODER_Y == True:
            navigationController.set_axis_PID_arguments(1, PID_P_Y, PID_I_Y, PID_D_Y)
            navigationController.configure_encoder(1, (SCREW_PITCH_Y_MM * 1000) / ENCODER_RESOLUTION_UM_Y,
                                                            ENCODER_FLIP_DIR_Y)
            navigationController.set_pid_control_enable(1, ENABLE_PID_Y)
        if HAS_ENCODER_Z == True:
            navigationController.set_axis_PID_arguments(2, PID_P_Z, PID_I_Z, PID_D_Z)
            navigationController.configure_encoder(2, (SCREW_PITCH_Z_MM * 1000) / ENCODER_RESOLUTION_UM_Z,
                                                            ENCODER_FLIP_DIR_Z)
            navigationController.set_pid_control_enable(2, ENABLE_PID_Z)
    time.sleep(0.5)

In [ ]:
# 设置软件限位
# set software limit
navigationController.set_x_limit_pos_mm(SOFTWARE_POS_LIMIT.X_POSITIVE)
navigationController.set_x_limit_neg_mm(SOFTWARE_POS_LIMIT.X_NEGATIVE)
navigationController.set_y_limit_pos_mm(SOFTWARE_POS_LIMIT.Y_POSITIVE)
navigationController.set_y_limit_neg_mm(SOFTWARE_POS_LIMIT.Y_NEGATIVE)
navigationController.set_z_limit_pos_mm(SOFTWARE_POS_LIMIT.Z_POSITIVE)

In [ ]:
# 移动检查
navigationController.move_x_to(8)
while microcontroller.is_busy():
    time.sleep(0.005)
navigationController.move_y_to(8)
while microcontroller.is_busy():
    time.sleep(0.005)
navigationController.home_z()
while microcontroller.is_busy():
    time.sleep(0.005)
navigationController.home_xy()
while microcontroller.is_busy():
    time.sleep(0.005)

In [ ]:
configurationManager = core.ConfigurationManager('./channel_configurations.xml')
from Drives.camera import Camera
if number_camera == 1:
    camera = Camera(rotate_image_angle=0)
    camera.open()
elif number_camera == 2:
    configuration1 = configurationManager.configurations[0]
    configuration2 = configurationManager.configurations[1]
    camera1 = Camera(sn=configuration1.camera_sn,rotate_image_angle=0)
    camera1.open()
    camera2 = Camera(sn=configuration2.camera_sn,rotate_image_angle=0)
    camera2.open()

In [ ]:
# 设置相机白平衡
if number_camera == 1:
    configuration = configurationManager.configurations[0]
    letters_only = re.sub(r'\d+', '', configuration.camera_sn)
    if letters_only == 'FCU':
        camera.set_wb_ratios(1,1,1)
    # 设置拍摄区域
    offset_x = (732 // 8) * 8
    offset_y = (238 // 8) * 8
    camera.set_ROI(offset_x, offset_y, 2560, 2560)
    # 设置曝光时间和增益
    camera.set_exposure_time(configuration.exposure_time)
    camera.set_analog_gain(configuration.analog_gain)
    # 启用软件触发拍摄
    camera.set_software_triggered_acquisition()
    camera.start_streaming()
    camera.send_trigger()
    img = camera.read_frame()
    if img is not None:
        print('相机检测成功')
elif number_camera == 2:
    letters_only = re.sub(r'\d+', '', configuration1.camera_sn)
    if letters_only == 'FCU':
        camera1.set_wb_ratios(1,1,1)
    # 设置拍摄区域
    offset_x = (732 // 8) * 8
    offset_y = (238 // 8) * 8
    camera1.set_ROI(offset_x, offset_y, 2560, 2560)
    # 设置曝光时间和增益
    camera1.set_exposure_time(configuration1.exposure_time)
    camera1.set_analog_gain(configuration1.analog_gain)
    # 启用软件触发拍摄
    camera1.set_software_triggered_acquisition()
    camera1.start_streaming()
    camera1.send_trigger()
    img = camera1.read_frame()
    if img is not None:
        print('相机1检测成功')
    letters_only = re.sub(r'\d+', '', configuration2.camera_sn)
    if letters_only == 'FCU':
        camera2.set_wb_ratios(1,1,1)
    # 设置拍摄区域
    offset_x = (732 // 8) * 8
    offset_y = (238 // 8) * 8
    camera2.set_ROI(offset_x, offset_y, 2560, 2560)
    # 设置曝光时间和增益
    camera2.set_exposure_time(configuration2.exposure_time)
    camera2.set_analog_gain(configuration2.analog_gain)
    # 启用软件触发拍摄
    camera2.set_software_triggered_acquisition()
    camera2.start_streaming()
    camera2.send_trigger()
    img = camera2.read_frame()
    if img is not None:
        print('相机2检测成功')

In [ ]:
###玻片装载器设备自检测试
from Drives.loadercontroller import LoaderController
oil = True
if oil :
    Loader = LoaderController(port = 'COM2', port_oil = 'COM3',True)
else :
    Loader = LoaderController(port = 'COM2', port_oil = 'COM3',False)

In [ ]:
# 装载器复位
Loader.reset_xyz()

In [ ]:
# 移动检查
Loader.set_delivery_abs_x(200)
Loader.set_delivery_abs_z(60)
Loader.set_delivery_abs_x(0)
Loader.set_delivery_abs_z(0)

In [ ]:
# 滴油测试
Loader.user_pump_test()